<a href="https://colab.research.google.com/github/rngadam/ART/blob/master/Retrieving_scientific_papers_from_CORE_on_LCAQS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CORE APIv3 
## Download all the works related to given query

https://api.core.ac.uk/docs/v3

In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
apikey=""
with open('drive/MyDrive/lcaqs/core-apikey.txt') as keyfile:
    apikey = keyfile.read().strip()
headers={"Authorization":"Bearer "+apikey}


In [3]:
import requests
def query_api(search_url, query, scrollId=None):
    if not scrollId:
        response = requests.get(f"{search_url}?q={query}&limit=100&scroll=true",headers=headers)
    else:
        response = requests.get(f"{search_url}?q={query}&limit=100&scrollId={scrollId}",headers=headers)        
    return response.json(), response.elapsed.total_seconds()

def scroll(search_url, query, extract_info_callback):
    allresults = []
    count = 0
    scrollId=None
    while True:
        result, elapsed = query_api(search_url, query, scrollId)
        if 'message' in result:
          print(result['message'])
          break
        result_size = len(result["results"])
        if "scrollId" in result:
          scrollId=result["scrollId"]
          totalhits = result["totalHits"]
        else:
          scrollId = None
          totalhits = result_size
        if result_size==0:
            break
        for hit in result["results"]:
            if extract_info_callback:
              allresults.append(extract_info_callback(hit))
            else:
              allresults.append(extract_info(hit))
        count += result_size
        print(f"{count}/{totalhits} {elapsed}s")
        if scrollId is None:
          break
    return allresults
        
def extract_info(hit):
    return {"id":hit["id"], "name": hit["name"], "url":hit["oaiPmhUrl"]}

In [4]:
def get_ids(hit):
  return {
      "id":hit["id"],
      "arxivId":hit["arxivId"],
      "doi":hit["doi"],
      "magId":hit["magId"],
      "coreIds":",".join(hit["outputs"]),
      "pubmedId":hit["pubmedId"]
  }


In [5]:
import json

response_filename = f'drive/MyDrive/lcaqs/query1.json'

In [6]:
import os

response_filename = f'drive/MyDrive/lcaqs/query1.json'
if not os.path.isfile(response_filename):
  response = scroll("https://api.core.ac.uk/v3/search/works", "(title:air%20quality AND sensor AND low-cost) AND year>=2017 AND _exists_:fullText", get_ids)
  with open(response_filename, 'w+') as json_file:
    json_file.write(json.dumps(response))
else:
  with open(response_filename) as response_file:
      response = json.loads(response_file.read())  

In [8]:
import os
import time

MAX_COUNT=24
count = 0
total_count = 0
for r in response:
  id = r['id']
  total_count+=1
  output_filename = f'drive/MyDrive/lcaqs/query1/{id}.json'
  if not os.path.isfile(output_filename):
    print(f'Retrieving {id}')
    singlework = requests.get(f'https://api.core.ac.uk/v3/search/works?q=id:{id}', headers=headers)
    if singlework.ok:
      with open(output_filename, 'w+') as json_file:
        json_file.write(singlework.text)
    else:
      raise Exception(singlework.reason)
    count += 1
    if count >= MAX_COUNT:
      time.sleep(360)
      count = 0
  else:
    print(f"already have {id}")

print(f"total {total_count}")


skipping 98538532
skipping 48994575
skipping 250861462
skipping 64229907
skipping 49986398
skipping 98157275
skipping 47687509
skipping 46933508
skipping 49676291
skipping 69389031
skipping 98083142
skipping 184311745
skipping 185313834
skipping 185310229
skipping 50332323
skipping 48954627
skipping 49818319
skipping 69402768
skipping 98558777
skipping 90345196
skipping 71349728
skipping 49130207
skipping 91187274
skipping 90596566
skipping 51321004
skipping 41848556
skipping 71385198
skipping 90294150
skipping 66505852
skipping 185313835
skipping 80786293
skipping 254176999
skipping 98382681
skipping 97915705
skipping 71861091
skipping 26217795
skipping 90329187
skipping 41840238
skipping 69390494
skipping 46869238
skipping 90534208
skipping 77840081
skipping 59142362
skipping 253255188
skipping 98557695
skipping 196635613
skipping 69390716
skipping 70228118
skipping 59541320
skipping 98760301
skipping 69358920
skipping 66058992
skipping 70233450
skipping 59024628
skipping 95166188
sk